In [2]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

In [10]:
# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "X9EGwt5zbpg8o5xk"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()

<LoginResource>

In [11]:
# Grab all event type ids. This will return a list which we will iterate over to print out the id and the name of the sport
event_types = trading.betting.list_event_types()

sport_ids = pd.DataFrame({
    'Sport': [event_type_object.event_type.name for event_type_object in event_types],
    'ID': [event_type_object.event_type.id for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Australian Rules,61420
Baseball,7511
Basketball,7522
Boxing,6
Cricket,4
Cycling,11
Darts,3503
Esports,27454571


In [12]:
# Filter for just horse racing
greyhound_racing_filter = betfairlightweight.filters.market_filter(text_query='Greyhound Racing')

# This returns a list
greyhound_racing_event_type = trading.betting.list_event_types(
    filter=greyhound_racing_filter)

# Get the first element of the list
greyhound_racing_event_type = greyhound_racing_event_type[0]

greyhound_racing_event_type_id = greyhound_racing_event_type.event_type.id
print(f"The event type id for greyhound racing is {greyhound_racing_event_type_id}")

The event type id for greyhound racing is 4339
